In [1]:
import numpy as np
import pandas as pd

import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import re

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer,SnowballStemmer

from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
# nltk.download('wordnet')
# nltk.download('stopwords')

from sklearn.metrics import classification_report
from sklearn import metrics

In [2]:
df_real_news_orig = pd.read_csv("../data/real_news.csv") # https://github.com/cuilimeng/CoAID/blob/master/07-01-2020/NewsRealCOVID-19.csv
df_fake_news_orig = pd.read_csv("../data/fake_news.csv") # https://github.com/cuilimeng/CoAID/blob/master/07-01-2020/NewsFakeCOVID-19.csv

df_real_news_orig = pd.DataFrame({"title": df_real_news_orig['title'] + df_real_news_orig['content'].replace(np.nan, ' NULL', regex=True)})
df_fake_news_orig = pd.DataFrame({"title": df_fake_news_orig['title'] + df_fake_news_orig['content'].replace(np.nan, ' NULL', regex=True)})

In [3]:
df_real_news = pd.concat([df_real_news_orig[['title']], pd.DataFrame({"label": np.ones((len(df_real_news_orig)), dtype=int)})], axis=1).reset_index(drop=True)[:len(df_fake_news_orig)]
df_fake_news = pd.concat([df_fake_news_orig[['title']], pd.DataFrame({"label": np.zeros((len(df_fake_news_orig)), dtype=int)})], axis=1).reset_index(drop=True)
df_news = pd.concat([df_real_news, df_fake_news], axis=0).reset_index(drop=True)
df_news.to_csv("../data/covid_news.csv", index=False)

In [4]:
# #Define function for removing special characters
# def remove_special_characters(text, remove_digits=True):
#     pattern=r'[^a-zA-z0-9\s]'
#     text=re.sub(pattern, " ", text)
#     return text

# #Stemming the text
# def simple_lemmatizer(text):
#     ps = WordNetLemmatizer()
#     text= ' '.join([ps.lemmatize(word.lower(), pos="v") for word in text.split()])
#     return text

# #set stopwords to english
# stop=set(stopwords.words('english'))
# # print(stop)

# #removing the stopwords
# def remove_stopwords(text, is_lower_case=True):
#     #Tokenization of text
#     tokenizer = ToktokTokenizer()
#     #Setting English stopwords
#     stopword_list = nltk.corpus.stopwords.words('english')

#     tokens = tokenizer.tokenize(text)
#     tokens = [token.strip() for token in tokens]
#     if is_lower_case:
#         filtered_tokens = [token for token in tokens if token not in stopword_list]
#     else:
#         filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]

#     filtered_text = ' '.join(filtered_tokens)    
#     return filtered_text

# def pipelinize(function, active=True):
#     def list_comprehend_a_function(list_or_series, active=True):
#         if active:
#             return [function(i) for i in list_or_series]
#         else: # if it's not active, just pass it right back
#             return list_or_series
#     return FunctionTransformer(list_comprehend_a_function, validate=False, kw_args={'active':active})

In [5]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
# nltk.download('punkt')

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [6]:
# estimators = [('remove_special_chars', pipelinize(remove_special_characters)),
#              ('simple_lemmatizer', pipelinize(simple_lemmatizer)),
#              ('remove_stopwords', pipelinize(remove_stopwords)),
#              ('count_vectorizer', CountVectorizer(ngram_range=(1,1)))]
CountVectorizer()
estimators = [
             ('count_vectorizer', CountVectorizer(ngram_range=(1,1), stop_words="english", lowercase=True, tokenizer=LemmaTokenizer()))]
pipeline = Pipeline(estimators)

In [7]:
df = pd.read_csv("../data/covid_news.csv")
X_train, X_test, y_train, y_test = train_test_split(df['title'], df['label'], test_size=0.2, random_state=42) 
X_train_all = pd.concat([X_train, y_train], axis=1)
X_test_all = pd.concat([X_test, y_test], axis=1)
X_train_all.to_csv("../data/train_news.csv", index=False)
X_test_all.to_csv("../data/test_news.csv", index=False)

In [8]:
X_train_ = pipeline.fit_transform(X_train)
X_test_ = pipeline.transform(X_test)

y_train_ = y_train
y_test_ = y_test

/Users/kelvinsoh/opt/anaconda3/envs/virtualenv/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [9]:
print('BOW_cv_train:', X_train_.shape)
print('BOW_cv_test:', X_test_.shape)

BOW_cv_train: (425, 3553)
BOW_cv_test: (107, 3553)


In [10]:
# estimators_tfidf = [('remove_special_chars', pipelinize(remove_special_characters)),
#              ('simple_lemmatizer', pipelinize(simple_lemmatizer)),
#              ('remove_stopwords', pipelinize(remove_stopwords)),
#              ('tfidf_vectorizer', TfidfVectorizer(use_idf=True, ngram_range=(1,1)))]
estimators_tfidf = [
             ('tfidf_vectorizer', TfidfVectorizer(use_idf=True, ngram_range=(1,1), stop_words="english", lowercase=True, tokenizer=LemmaTokenizer()))]
pipeline_tfidf = Pipeline(estimators_tfidf)

X_train_tfidf = pipeline_tfidf.fit_transform(X_train)
X_test_tfidf = pipeline_tfidf.transform(X_test)

y_train_tfidf = y_train
y_test_tfidf = y_test

In [11]:
y_test_tfidf.value_counts()

1    59
0    48
Name: label, dtype: int64

In [12]:
y_train_tfidf.value_counts()

0    218
1    207
Name: label, dtype: int64

In [13]:
print('Tfidf_train:',X_train_tfidf.shape)
print('Tfidf_test:',X_test_tfidf.shape)

Tfidf_train: (425, 3553)
Tfidf_test: (107, 3553)


### LogisticRegression (BOW)

In [14]:
lr = LogisticRegression(penalty='l2',max_iter=1000,C=1,random_state=42)
lr_bow = lr.fit(X_train_, y_train_)

lr_bow_predict = lr.predict(X_test_)
accuracy_score(lr_bow_predict, y_test_)

0.9158878504672897

### LogisticRegression (TFIDF)

In [15]:
lr_tfidf = LogisticRegression(penalty='l2',max_iter=1000, C=1, random_state=42)
lr_tfidf.fit(X_train_tfidf, y_train_tfidf)

lr_tfidf_predict = lr_tfidf.predict(X_test_tfidf)
accuracy_score(lr_tfidf_predict, y_test_tfidf)

0.9345794392523364

### Simple Prediction (LogisticRegression (TFIDF))

In [16]:
input_text = ["Real news for donald trump NULL covid context cdc facebook", "Hello how are you?", "What New CDC Guidelines Mean for Workplaces as They Reopenexperts say business owners should adopt safety practices that best fit their workplace during the covid-19 pandemic. getty images experts say there are many challenges to reopening workplaces during the covid-19 pandemic even with the release of federal guidelines. they advise business owners to use the new recommendations simply as guideposts and to adopt measures that best fit their workplace and community. making employees feel safe is a key ingredient to a successful reopening say ex. "]
input_text_transform = pipeline_tfidf.transform(input_text)
lr_tfidf.predict_proba(np.array(input_text_transform.todense()))

array([[0.71342251, 0.28657749],
       [0.57705878, 0.42294122],
       [0.26054912, 0.73945088]])

### RandomForestClassifier (TFIDF)

In [17]:
rf_tfidf = RandomForestClassifier(random_state=42)
rf_tfidf.fit(X_train_tfidf, y_train_tfidf)

rf_tfidf_predict = rf_tfidf.predict(X_test_tfidf)
accuracy_score(rf_tfidf_predict, y_test_tfidf)

0.9345794392523364

### Confusion Matrix (LogisticRegression TFIDF)

In [18]:
confusion_matrix(lr_tfidf_predict, y_test_)

array([[43,  2],
       [ 5, 57]])

### Classification Report (LogisticRegression TFIDF)

In [19]:
print(classification_report(y_test_, lr_tfidf_predict))

              precision    recall  f1-score   support

           0       0.96      0.90      0.92        48
           1       0.92      0.97      0.94        59

    accuracy                           0.93       107
   macro avg       0.94      0.93      0.93       107
weighted avg       0.94      0.93      0.93       107



### AUC (LogisticRegression TFIDF)

In [20]:
fpr, tpr, thresholds = metrics.roc_curve(np.array(y_test_tfidf), lr_tfidf.predict_proba(X_test_tfidf)[:, 1])
metrics.auc(fpr, tpr)

0.9766949152542372